# Idea 1
- We induce subgraph of proteins (or maybe subgraph of proteins + 1 degree of separation away)
- Find communities using MCL or Louvain to get us communities of size ~30
- Determine if these communities are functional
- Find bottlenecks/hub proteins of each complex
- This gives us relevant proteins for treatment or disrupting pathways

In [ ]:
import networkx as nx
import numpy as np
import scipy as sp
import pandas as pd

In [ ]:
# Reading in graph 
G = nx.read_weighted_edgelist("yeast.txt",comments="#",nodetype=str)

Removing edges not meeting threshold score. **Need to decide what we are doing with essential nodes.**

In [ ]:
# Deleting edges that don't meet threshold score
threshold_score = 700
for edge in G.edges: 
    weight = list(G.get_edge_data(edge[0],edge[1]).values())
    if(weight[0] <= threshold_score):
        G.remove_edge(edge[0],edge[1])

Relabelling nodes to get rid of 4932 tag

In [ ]:
H = nx.relabel_nodes(G, lambda x: x[5:])

Getting list of all yeast homologs

In [ ]:
df = pd.read_csv("human_to_yeast.csv")
homologs = list(set(list(df["homolog_systematic_name"])))

Getting subgraph induced by yeast homologs

In [ ]:
H0 = H.subgraph(homologs).copy()

In [ ]:
print(H0)

Louvain

In [ ]:
louvain = nx.algorithms.community.louvain_communities(H0, resolution=7, seed=123)
louvain.sort(key=len, reverse=True)

number_of_communities = len(louvain)
size_of_communities = [len(community) for community in louvain]

print("Number of communities: {}".format(number_of_communities))
print("Sizes of communities: ", size_of_communities)

Drawing communities (taking the t biggest communities)

In [ ]:
import matplotlib.pyplot as plt
from netgraph import Graph
import random

In [ ]:
# parameter that controls how many communities we are interested in
t = 3

# Generating list of nodes of first t communities
nodes = []
for i in range(t):
  nodes.extend(louvain[i]) 


# Subgraph induced by these communities
H1 = H0.subgraph(nodes).copy()

# Dictionary where key is node and value is community index
node_to_community = {}
for i in range(t):
  for node in louvain[i]:
    node_to_community[node] = i

# Assigning t random colours each of our communities
community_to_color = dict([(i, "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])) for i in range(t)])
node_color = {node: community_to_color[community_id] for node, community_id in node_to_community.items()}

plt.figure(figsize=(20,20))
Graph(H1,
      node_color=node_color, node_edge_width=0, edge_alpha=0.1, node_size = 0.5, edge_width = 0.5,
      node_layout='community', node_layout_kwargs=dict(node_to_community=node_to_community),
      edge_layout='bundled', edge_layout_kwargs=dict(k=2000)
)

plt.show()

## Centrality

The sections below find the most central protein in each community, where each entry in the list is {community_index: (protein, centrality_value)}.

Degree Centrality

In [ ]:
hub = {}
for index, community in enumerate(louvain):
  sub = H0.subgraph(community).copy()
  d = sorted(list(nx.algorithms.centrality.degree_centrality(sub).items()), key=lambda x: x[1], reverse=True)
  hub[index] = d[0]

print(hub)

Eigenvector Centrality

In [ ]:
hub = {}
for index, community in enumerate(louvain):
  sub = H0.subgraph(community).copy()
  d = sorted(list(nx.algorithms.centrality.eigenvector_centrality(sub).items()), key=lambda x: x[1], reverse=True)
  hub[index] = d[0]

print(hub)

Betweenness Centrality

In [ ]:
hub = {}
for index, community in enumerate(louvain):
  sub = H0.subgraph(community).copy()
  d = sorted(list(nx.algorithms.centrality.betweenness_centrality(sub).items()), key=lambda x: x[1], reverse=True)
  hub[index] = d[0]

print(hub)

VoteRank

In [ ]:
print(nx.algorithms.centrality.voterank(H0, 10))

Creating graph of where each community is a node and the weight of edges between two communities A and B is equal to the number of communities between A and B.

In [ ]:
# Going through all communities and setting the node attribute to be the index of the community it is in
community_dict = {}
for index, community in enumerate(louvain):
  for node in community:
    community_dict[node] = index

nx.set_node_attributes(H0, community_dict, "community")


# Creating basic graph with all our communities as nodes, but no edges yet
F = nx.Graph()
F.add_nodes_from(range(0, number_of_communities))

for (u, v) in H0.edges:
  community_i = H0.nodes[u]["community"]
  community_j = H0.nodes[v]["community"]
  
  # if in different communities
  if community_i != community_j:
    
    # if community graph doesnt already have edge, we have to add the edge
    if not F.has_edge(community_i, community_j):
      F.add_edge(community_i, community_j, weight = 1)
    else:
      F[community_i][community_j]["weight"] += 1

Finding max weight (min weight is trivially 1)

In [ ]:
max_weight = 1
for edge in F.edges:
  if F.edges[edge]["weight"] > max_weight:
    max_weight = F.edges[edge]["weight"]

print(max_weight)

Betweenness Centrality

Output below tells us that community 26 has the highest betweenness centrality (so it could be involved in many interactions etc.)

In [ ]:
d = list(nx.algorithms.centrality.betweenness_centrality(F).items())
print(sorted(d, reverse=True, key=lambda x: x[1]))

In [ ]:
# Relating node sizes in the plot to how central the communities are (i.e. how large their betweenness centrality is).
# Note that communities with zero centrality disappear from the plot (aka communities which are not connected)
node_size = []
for i in range(len(d)):
  if d[i][1] != 0:
    node_size.append(d[i][1] * 200000)
nodelist = [node for (node, c) in d if c != 0]

# Relating edge opacity to weight.
edgelist = [(u, v) for (u, v) in F.edges if u in nodelist and v in nodelist]
edge_color = []
for edge in edgelist:
  # We normalise weight value so that it becomes between 0-1 for alpha values in RGBA
  edge_color.append((0, 0, 0, (F.edges[edge]["weight"] - 1) / (max_weight - 1)))

# Adding weight labels (bc why not, this is easily removed)
labels = nx.get_edge_attributes(F,'weight')

# Scaling node labels with node sizes


plt.figure(figsize=(20,20))
pos=nx.spring_layout(F)
nx.draw_networkx(F, pos, nodelist=nodelist, edgelist=edgelist, node_size=node_size, edge_color=edge_color, width=3)

# Adding weight labels (bc why not, this is easily removed)
# labels = nx.get_edge_attributes(F,'weight')
# nx.draw_networkx_edge_labels(F, pos, edge_labels=labels, font_size=5)

# TODO Scaling node labels with node size

plt.show()